In [1]:
# Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Librerie
import os, time, random
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from collections import Counter

# Imposta seed e device
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type=='cuda':
    torch.cuda.manual_seed(42)
print(f"Using device: {device}")

# 1) PERCORSI AI DATI ORGANIZZATI
TENT_ROOT = Path("/content/drive/MyDrive/rumination_project/tentativo")
train_root_dirs = [
    TENT_ROOT/"video1"/"train",
    TENT_ROOT/"video2"/"train"
]
val_root_dirs = [
    TENT_ROOT/"video1"/"val",
    TENT_ROOT/"video2"/"val"
]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda


In [2]:
# 2) RACCOGLI PATH E LABEL, CONTEGGIA CLASSI
def gather(paths_root):
    paths, labels = [], []
    for root in paths_root:
        pos_dir = root/"pos"
        neg_dir = root/"neg"
        for p in pos_dir.rglob("*.jpg"):
            paths.append(str(p)); labels.append(1)
        for p in neg_dir.rglob("*.jpg"):
            paths.append(str(p)); labels.append(0)
    return paths, labels

train_paths, train_labels = gather(train_root_dirs)
val_paths,   val_labels   = gather(val_root_dirs)

print(f" Train images: {len(train_paths)}; class dist: {Counter(train_labels)}")
print(f" Val   images: {len(val_paths)}; class dist: {Counter(val_labels)}")

# 3) TRANSFORMS
train_tf = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomResizedCrop(224, scale=(0.7,1.0), ratio=(0.8,1.2)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.2),
    transforms.RandomRotation(20),
    transforms.ColorJitter(0.4,0.4,0.4,0.1),
    transforms.RandomGrayscale(0.1),
    transforms.RandomAffine(15, translate=(0.1,0.1), scale=(0.9,1.1), shear=10),
    transforms.GaussianBlur(3, sigma=(0.1,2.0)),
    transforms.RandomPerspective(0.3),
    transforms.ToTensor(),
    transforms.RandomErasing(0.3, scale=(0.02,0.15), ratio=(0.3,3.3)),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

 Train images: 2043; class dist: Counter({0: 1243, 1: 800})
 Val   images: 2044; class dist: Counter({0: 1244, 1: 800})


In [3]:
# 4) DATASET
class SimpleDataset(Dataset):
    def __init__(self, paths, labels, tf):
        self.paths, self.labels, self.tf = paths, labels, tf
    def __len__(self): return len(self.paths)
    def __getitem__(self, i):
        img = Image.open(self.paths[i]).convert("RGB")
        return self.tf(img), torch.tensor(self.labels[i], dtype=torch.long)

train_ds = SimpleDataset(train_paths, train_labels, train_tf)
val_ds   = SimpleDataset(val_paths,   val_labels,   val_tf)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,  num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=2)

# 5) MODELLO ResNet-18
class ResNetClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(pretrained=True)
        in_f = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_f,256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.Dropout(0.25),
            nn.Linear(256,2)
        )
    def forward(self,x): return self.backbone(x)

model = ResNetClassifier().to(device)

# 6) LOSS con pesi di classe dinamici
class_counts = Counter(train_labels)
total = sum(class_counts.values())
class_weights = torch.tensor([
    total/class_counts[0],
    total/class_counts[1]
], device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)

optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=8)

# 7) TRAINING + EARLY STOPPING
NUM_EPOCHS = 10
PATIENCE   = 5
best_f1    = 0.0
no_imp     = 0

for epoch in range(1, NUM_EPOCHS+1):
    # Train
    model.train()
    tr_preds, tr_trues, tr_loss = [], [], 0.0
    for imgs, labs in train_loader:
        imgs, labs = imgs.to(device), labs.to(device)
        optimizer.zero_grad()
        outs = model(imgs)
        loss = criterion(outs, labs)
        loss.backward(); optimizer.step()
        tr_loss += loss.item()
        tr_preds += outs.argmax(1).cpu().tolist()
        tr_trues += labs.cpu().tolist()
    tr_acc = accuracy_score(tr_trues, tr_preds)

    # Val
    model.eval()
    v_preds, v_trues, v_probs, v_loss = [], [], [], 0.0
    with torch.no_grad():
        for imgs, labs in val_loader:
            imgs, labs = imgs.to(device), labs.to(device)
            outs = model(imgs)
            v_loss += criterion(outs, labs).item()
            prob = torch.softmax(outs,1)[:,1].cpu().tolist()
            v_probs += prob
            v_preds += outs.argmax(1).cpu().tolist()
            v_trues += labs.cpu().tolist()
    v_acc = accuracy_score(v_trues, v_preds)
    v_f1  = f1_score(v_trues, v_preds, zero_division=0)
    v_auc = roc_auc_score(v_trues, v_probs)

    print(f"Epoch {epoch}/{NUM_EPOCHS} | "
          f"TrL={tr_loss/len(train_loader):.4f} TrA={tr_acc:.3f} | "
          f"ValL={v_loss/len(val_loader):.4f} ValA={v_acc:.3f} F1={v_f1:.3f} AUC={v_auc:.3f}")

    scheduler.step(v_f1)

    # Early stopping
    if v_f1 > best_f1:
        best_f1 = v_f1; no_imp = 0
        torch.save(model.state_dict(), "/content/drive/MyDrive/best_resnet18.pth")
    else:
        no_imp += 1
        if no_imp >= PATIENCE:
            print(f" Early stopping at epoch {epoch}")
            break

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 177MB/s]


Epoch 1/10 | TrL=0.6898 TrA=0.646 | ValL=0.5679 ValA=0.782 F1=0.728 AUC=0.841


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 2/10 | TrL=0.5987 TrA=0.709 | ValL=0.4899 ValA=0.819 F1=0.765 AUC=0.896


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


...

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 6/10 | TrL=0.4457 TrA=0.846 | ValL=0.5392 ValA=0.778 F1=0.645 AUC=0.907


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 7/10 | TrL=0.4442 TrA=0.851 | ValL=0.4680 ValA=0.821 F1=0.752 AUC=0.917
 Early stopping at epoch 7


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


In [4]:
# 8) CONFUSION MATRIX FINALE
model.load_state_dict(torch.load("/content/drive/MyDrive/best_resnet18.pth", map_location=device))
model.eval()
all_preds, all_trues = [], []
with torch.no_grad():
    for imgs, labs in val_loader:
        imgs, labs = imgs.to(device), labs.to(device)
        outs = model(imgs)
        all_preds += outs.argmax(1).cpu().tolist()
        all_trues += labs.cpu().tolist()

cm = confusion_matrix(all_trues, all_preds)
print(" Confusion Matrix on VALIDATION SET:\n", cm)

 Confusion Matrix on VALIDATION SET:
 [[1073  171]
 [ 198  602]]
